# **Yelp Data Predictive Modeling**

Joseph Lardie

December 2023

# **Objective:**

- Predicting the ratings for Brooklyn Pizza Restaurants on YELP

# **Imports**

In [1]:
#Numpy
import numpy as np

#Pandas
import pandas as pd

#Seaborn
import seaborn as sns

#matplotlib
import matplotlib.pyplot as plt
import plotly
%matplotlib inline

# Standard imports
import re

#sklearn
from sklearn import preprocessing
from sklearn import set_config
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer 
from sklearn.metrics import r2_score, confusion_matrix, roc_auc_score, roc_curve, classification_report, mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# warnings
import warnings
warnings.filterwarnings("ignore")
set_config(display = 'diagram')

# **Loading Data**

In [2]:
# Loading YELP Review Data
ydf = pd.read_csv('ydf')

In [3]:
# Glance of YELP Data
ydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   alias         847 non-null    object 
 1   name          847 non-null    object 
 2   image_url     847 non-null    object 
 3   url           847 non-null    object 
 4   review_count  847 non-null    int64  
 5   categories    847 non-null    object 
 6   rating        847 non-null    float64
 7   coordinates   847 non-null    object 
 8   transactions  847 non-null    object 
 9   location      847 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 66.3+ KB


# **Data Cleaning**

In [4]:
# Dropping irrelevant 
ydf.drop(['image_url', 'url'], axis=1, inplace=True)

# **Validation Split**

In [5]:
# Defining the target variable
X= ydf.drop('rating', axis=1)
y= ydf['rating']

# **Data Preprocessing**

In [6]:
# Split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Check for missing values
print("Missing values in X_train:", X_train.isnull().sum().sum())
print("Missing values in y_train:", y_train.isnull().sum().sum())

# Check data types
print("Data types in X_train:\n", X_train.dtypes)
print("Data type in y_train:", y_train.dtypes)

Missing values in X_train: 0
Missing values in y_train: 0
Data types in X_train:
 alias           object
name            object
review_count     int64
categories      object
coordinates     object
transactions    object
location        object
dtype: object
Data type in y_train: float64


In [8]:
#Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy= 'mean')

# Standard Scaler
scaler = StandardScaler()

# OneHotEncoder
ohe = OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore')

# Instantiate the selectors
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

# Forming pipelines
num_pipe = make_pipeline(mean_imputer, scaler)
cat_pipe = make_pipeline(freq_imputer, ohe)

#Create Tuples
num_tuple = ('num', num_pipe, num_selector)
cat_tuple = ('cat', cat_pipe, cat_selector)

#Make tuples for preprocessing the categorical and numeric columns
num_tuple = (num_pipe, num_selector)
cat_tuple = (cat_pipe, cat_selector)  

#Create Column Transformer
preprocessor= make_column_transformer(num_tuple, cat_tuple, remainder='passthrough')

In [9]:
# Looking at how the preprocessor is organized
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000247F0BA32E0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000247F0BA3640>)])

# **Linear Regression**

In [10]:
# Create the final pipeline with the preprocessor and the linear regression model
lr_model = make_pipeline(preprocessor, LinearRegression())

In [11]:
# Train the model
lr_model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000247F0BA32E0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000247F0BA3640>)])),
                ('linearregression', LinearRegression())])

In [12]:
# Make predictions on the training set
y_train_pred = lr_model.predict(X_train)

# Make predictions on the test set
y_test_pred = lr_model.predict(X_test)

In [13]:
# Evaluate the model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

print(f'Training RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

Training RMSE: 5.440084453739275e-15
Test RMSE: 0.4696251860438544


In [18]:
# Scatter plot of actual vs. predicted values
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test_pred , y=y_train_pred)
plt.xlabel('Actual Ratings')
plt.ylabel('Predicted Ratings')
plt.title('Linear Regression: Actual vs. Predicted Ratings')
plt.show()

ValueError: All arrays must be of the same length

<Figure size 800x600 with 0 Axes>

In [ ]:
# Residual plot
residuals = y_test - y_pred_lr
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Actual Ratings')
plt.ylabel('Residuals')
plt.title('Linear Regression: Residual Plot')
plt.show()

In [ ]:
# Distribution of residuals
plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Linear Regression: Distribution of Residuals')
plt.show()

In [ ]:
# Mean Squared Error
mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f'Mean Squared Error (Linear Regression): {mse_lr:.2f}')

# **KNN Model**

In [14]:
# Create the k-Nearest Neighbors model
knn_model = KNeighborsRegressor(n_neighbors=5)  # You can adjust the number of neighbors (n_neighbors) as needed

# Create the final pipeline with the preprocessor and the k-Nearest Neighbors model
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('knn', knn_model)])

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate on Test Set
# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# R-squared
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

Mean Squared Error: 0.2615882352941176
R-squared: 0.007919687674288856


In [ ]:
# Scatter plot of actual vs. predicted values
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred_knn)
plt.xlabel('Actual Ratings')
plt.ylabel('Predicted Ratings')
plt.title('KNN Regression: Actual vs. Predicted Ratings')
plt.show()

In [ ]:
# Residual plot
residuals_knn = y_test - y_pred_knn
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=residuals_knn)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Actual Ratings')
plt.ylabel('Residuals')
plt.title('KNN Regression: Residual Plot')
plt.show()

In [ ]:
# Distribution of residuals
plt.figure(figsize=(8, 6))
sns.histplot(residuals_knn, kde=True)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('KNN Regression: Distribution of Residuals')
plt.show()

In [ ]:
# Mean Squared Error
mse_knn = mean_squared_error(y_test, y_pred_knn)
print(f'Mean Squared Error (KNN Regression): {mse_knn:.2f}')